<a href="https://colab.research.google.com/github/semhoun/omnius/blob/main/nb/Claire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Acknowledgments
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" height="50"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" height="50"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" height="50"></a> <a href="https://github.com/unslothai/unsloth"><img src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" height="50"></a>
</div>

# Data Sources:
* https://huggingface.co/angeluriot
* https://huggingface.co/CATIE-AQ
* https://huggingface.co/jpacifico
* https://huggingface.co/OpenLLM-France

# Definitions


In [1]:
import os

if "COLAB_" in "".join(os.environ.keys()):
  from google.colab import userdata
  hf_token = userdata.get('HuggingFaceToken')
else:
  hf_token = os.environ["HUGGINGFACETOKEN"]

hf_username = "nsemhoun"
debug = True
save_hf = True
version = '3B'

if version == '9B':
  source_model = "nvidia/NVIDIA-Nemotron-Nano-9B-v2"
  quantization_method = ["q8_0", "f16", "q4_k_m", "q4_0"]
  model_name = "Claire-9B-v0.1.0"
else:
  #source_model = "HuggingFaceTB/SmolLM3-3B"
  source_model = "HuggingFaceTB/SmolLM3-3B-Base"
  quantization_method = ["q8_0", "f16"]
  model_name = "Claire-3B-v0.1.4"

# Installation

In [2]:
%%capture
import os, re
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

if "COLAB_" in "".join(os.environ.keys()):
    # Google colab
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --upgrade --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps --upgrade unsloth
    !pip install transformers==4.55.4
    !pip install --no-deps trl==0.22.2
elif "VAST_" in "".join(os.environ.keys()):
    # Vast.ai Unsloth version
    pass
else:
    !pip install --upgrade unsloth-zoo
    !pip install --upgrade unsloth
    !pip install transformers==4.55.4
    !pip install --no-deps trl==0.22.2

# Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = source_model,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = False,
    load_in_8bit = False,
    token = hf_token,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
from unsloth import FastModel

lora_rank = 16 # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      #"embed_tokens", "lm_head", # Add for continual pretraining if not tie_word_embeddings = True,
                      ],
    lora_alpha = lora_rank,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

# Dataset Tools


## Prompt formatter
**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

ChatML renders multi turn conversations like below:

```
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What's the capital of France?<|im_end|>
<|im_start|>assistant
Paris.
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old` and our own optimized `unsloth` template.

Normally one has to train `<|im_start|>` and `<|im_end|>`. We instead map `<|im_end|>` to be the EOS token, and leave `<|im_start|>` as is. This requires no additional training of additional tokens.

More info on chat templates on [our wiki page!](https://github.com/unslothai/unsloth/wiki#chat-templates)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [18]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template
import pprint, json
EOS_TOKEN = tokenizer.eos_token

wikipedia_prompt = """{} Article
### Title: {}

### Article:
{}"""

ebook_prompt = """Epub Book
### Filename: {}

### Title: {}

### Author: {}

### Subject: {}

### Part: {}

### Content:
{}"""

gutenberg_prompt = """Book
### Title: {}

### Author: {}

### Content:
{}"""

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style if needed
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples, kind):
  # Must add EOS_TOKEN, otherwise your generation will go on forever!

  match kind:
    case 'share_gpt':
      if (examples.get('conversations') != None):
        convos = examples["conversations"]
        texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
        return { "text" : texts, }
      if (examples.get('messages') != None):
        convos = examples["messages"]
        texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
        return { "text" : texts, }
      raise ValueError('Unknow sharegtp data format.')

    case 'share_gpt_context':
        texts = []
        for convo in examples["conversation"]:
          if len(convo.get('context')) == 0:
            texts.append(tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False))
          else
            row = [
                {"role": "system", "content": "Utilise le contexte entre <ctx> pour répondre. Si tu ne sais pas, dis-le.\n<ctx>\n" + convo.get('context') +"\n</ctx>"}
            ]
            row.append(convo.get('conversation'))
            texts.append(tokenizer.apply_chat_template(row, tokenize = False, add_generation_prompt = False))
        return { "text" : texts, }

    case 'wikipedia':
      sources = examples["sources"]
      titles = examples["title"]
      texts  = examples["text"]
      outputs = []
      for source, title, text in zip(sources, titles, texts):
          text = wikipedia_prompt.format(source.title(),title, text) + EOS_TOKEN
          outputs.append(text)
      return { "text" : outputs, }

    case 'gutenberg':
      titles = examples["title"]
      authors = examples["author"]
      texts  = examples["text"]
      outputs = []
      for title, author, text in zip(titles, authors,  texts):
        author = json.loads(author)
        text = gutenberg_prompt.format(title, author['author'], text) + EOS_TOKEN
        outputs.append(text)
      return { "text" : outputs, }

    case 'ebook':
      filenames = examples["filename"]
      titles = examples["title"]
      authors = examples["author"]
      subjects = examples["subject"]
      parts = examples["part"]
      contents  = examples["content"]
      outputs = []
      for filename, title, author, subject, part, content in zip(filenames, titles, authors, subjects, parts, contents):
        text = ebook_prompt.format(filename, title, author, subject, part, content) + EOS_TOKEN
        outputs.append(text)
      return { "text" : outputs, }

    case 'dpo':
      texts = []
      for convo in examples:
        if (convo.get('system') == None):
          transformed_data = [
              {"role": "user", "content": convo.get('question')},
              {"role": "assistant", "content": convo.get('chosen')},
          ]
        else:
          transformed_data = [
              {"role": "system", "content": convo.get('system')},
              {"role": "user", "content": convo.get('question')},
              {"role": "assistant", "content": convo.get('chosen')},
          ]
        texts.append(tokenizer.apply_chat_template(transformed_data, tokenize = False, add_generation_prompt = False))
      return { "text" : text, }

  raise ValueError('Unknow data format.')
pass

Unsloth: Will map <|im_end|> to EOS = <|im_end|>.


## Dataset helper

In [27]:
import random
from datasets import concatenate_datasets
from datasets import Dataset

def dumpDatasets(datasets):
  for key, value in datasets.items():
    print(key.upper() + '=' * 50)
    row = value[random.randrange(0, len(value), 1)]
    pprint.pp(row["text"])
pass

def mergeDatasets(datasets):
  resDataSet = Dataset.from_dict({})
  for key, value in datasets.items():
    if debug:
      value = value.train_test_split(train_size = 0.02)['train']
    resDataSet = concatenate_datasets([resDataSet, value])
  resDataSet = resDataSet.shuffle(seed=0)
  return resDataSet
pass


# Continued Pretraining

## Datasets(s)



In [7]:
#@title Init datasets
datasets = {}

In [8]:
#@title EBook data
dataset = load_dataset('nsemhoun/ebooks', split = 'train', token=hf_token)
dataset = dataset.map(formatting_prompts_func, batched = True, fn_kwargs= {'kind':'ebook'})
dataset = dataset.select_columns(['text'])
datasets['ebooks'] = dataset

books.jsonl:   0%|          | 0.00/2.08G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/108987 [00:00<?, ? examples/s]

Map:   0%|          | 0/108987 [00:00<?, ? examples/s]

In [ ]:
#@title Wikipedia data
dataset = load_dataset("OpenLLM-France/wikimedia", "fr", split = 'train')
dataset = dataset.map(formatting_prompts_func, batched = True, fn_kwargs= {'kind':'wikipedia'})
dataset = dataset.select_columns(['text'])
datasets['wikipedia'] = dataset

In [ ]:
#@title Gutenberg data
dataset = load_dataset("OpenLLM-France/Lucie-Training-Dataset", 'Gutenberg-fr', split = 'train',)
dataset = dataset.map(formatting_prompts_func, batched = True, fn_kwargs= {'kind':'gutenberg'})
dataset = dataset.select_columns(['text'])
datasets['gutenberg'] = dataset

In [9]:
#@title Merge datasets
dumpDatasets(datasets)
dataset = mergeDatasets(datasets)

EBOOKS==================================================
('Epub Book\n'
 '### Filename: crime du golf, Le - Agatha Christie.epub\n'
 '\n'
 '### Title: Le crime du golf\n'
 '\n'
 '### Author: Agatha Christie\n'
 '\n'
 '### Subject: Policier\n'
 '\n'
 '### Part: Christie,Agatha-Le crime du '
 'golf(1923).French.ebook.AlexandriZ_split_024\n'
 '\n'
 '### Content:\n'
 '24  Sauvez\\-le !\n'
 '================\n'
 '\n'
 'Nous revînmes en France par le bateau\n'
 'du soir, pour nous trouver le lendemain matin à Saint\\-Omer, où l’on avait\n'
 'transféré Jack Renauld. Poirot voulut aussitôt rendre visite à M. Hautet. '
 'Comme\n'
 'il ne semblait voir aucune objection à ma présence, je l’accompagnai.\n'
 '\n'
 'Après diverses formalités, nous\n'
 'fûmes introduits dans le bureau du magistrat, qui nous accueillit avec\n'
 'cordialité.\n'
 '\n'
 '— Je m’étais laissé dire que\n'
 'vous étiez retourné en Angleterre, monsieur Poirot. Je me réjouis de '
 'constater\n'
 'qu’il n’en est rien.\n'
 '\n'


## Training
Now let's use Unsloth's `UnslothTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer).
Also set `embedding_learning_rate` to be a learning rate at least 2x or 10x smaller than `learning_rate` to make continual pretraining work!

In [10]:
from transformers import TrainingArguments
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,

        # Use warmup_ratio and num_train_epochs for longer runs!
        max_steps = 60 if debug else -1, # None for full run
        num_train_epochs = 0 if debug else 1,
        warmup_ratio = 0.1,

        # Select a 2 to 10x smaller learning rate for the embedding matrices!
        learning_rate = 5e-5,
        embedding_learning_rate = 1e-5,

        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/2179 [00:00<?, ? examples/s]

In [11]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
5.857 GB of memory reserved.


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,179 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 30,228,480 of 3,105,327,104 (0.97% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.137300
2,2.030200
3,2.114100
4,2.088000
5,2.046700
6,2.038100
7,2.068200
8,2.097600
9,2.100900
10,2.074100


In [13]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

339.1487 seconds used for training.
5.65 minutes used for training.
Peak reserved memory = 8.641 GB.
Peak reserved memory for training = 2.784 GB.
Peak reserved memory % of max memory = 21.844 %.
Peak reserved memory for training % of max memory = 7.038 %.


# SFT (Chat)

## Datasets(s)

In [14]:
#@title Init datasets
datasets = {}

In [ ]:
#@title Orca dpo paris
dataset = load_dataset("jpacifico/french-orca-dpo-pairs-revised", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True, fn_kwargs= {'kind':'dpo'})
datasets['french-orca-dpo'] = dataset

In [15]:
#@title SmolTalk Think
dataset = load_dataset("CATIE-AQ/smoltalk2_aya_think_dataset_french_split", "french_raisonning", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True, fn_kwargs= {'kind':'share_gpt'})
datasets['smaltalk-think'] = dataset

README.md: 0.00B [00:00, ?B/s]

french/train-00000-of-00001.parquet:   0%|          | 0.00/3.85M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1393 [00:00<?, ? examples/s]

Map:   0%|          | 0/1393 [00:00<?, ? examples/s]

In [16]:
#@title SmolTalk ToolCalling
dataset = load_dataset("CATIE-AQ/smoltalk2_smolagents_toolcalling_french", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True, fn_kwargs= {'kind':'share_gpt'})
datasets['smaltalk-toolcalling'] = dataset

README.md:   0%|          | 0.00/968 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/82.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9079 [00:00<?, ? examples/s]

Map:   0%|          | 0/9079 [00:00<?, ? examples/s]

In [20]:
#@title French instruct
dataset = load_dataset("angeluriot/french_instruct", split = "train")
dataset = dataset.map(formatting_prompts_func, n_kwargs= {'kind':'share_gpt'})
datasets['french_instruct'] = dataset

TypeError: Dataset.map() got an unexpected keyword argument 'n_kwargs'

In [25]:
#@title Facebook Community
dataset = load_dataset("CATIE-AQ/facebook-community-alignment-dataset_french_conversation", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True, fn_kwargs= {'kind':'share_gpt'})
datasets['facebook'] = dataset

In [28]:
#@title Merge datasets
dumpDatasets(datasets)
dataset = mergeDatasets(datasets)

SMALTALK-THINK==================================================
('<|im_start|>user\n'
 'Qui était l’artiste Shuvinai Ashoona ?<|im_end|>\n'
 '<|im_start|>assistant\n'
 '<think>\n'
 "D'accord, l'utilisateur pose des questions sur Shuvinai Ashoona, une "
 "artiste. Permettez-moi de commencer par rappeler ce que je sais. C'est une "
 "artiste inuite du Canada, n'est-ce pas ? Je pense qu'elle vient de Cape "
 'Dorset, qui est connue pour son art inuit. Elle fait partie des Kinngait '
 'Studios, qui sont un lieu important pour les artistes inuits. Je me souviens '
 "qu'elle travaille avec la gravure et le dessin, souvent en utilisant des "
 'matériaux traditionnels comme la pierre et le papier.  Attendez, elle est '
 'également connue pour ses œuvres vibrantes et détaillées. Peut-être '
 'utilise-t-elle des thèmes inuits traditionnels, mais dans un style '
 'contemporain. Je devrais vérifier si elle fait partie de mouvements notables '
 "ou si elle a des œuvres célèbres. Oh, c'est vrai, el

## Train the model


Setup the SFT traincer with approprite arguments.

In [29]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, #1
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 60 if debug else -1, # None for full run
        num_train_epochs = 0 if debug else 1,
        learning_rate = 2e-4, # Lower for slower but more precise fine-tuning. Try values like 1e-4, 5e-5, or 2e-5
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # remove to activate WandDB
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/436 [00:00<?, ? examples/s]

## Training Execution
Execute the training process with the configured trainer and monitor the training progress.

In [30]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
8.641 GB of memory reserved.


In [31]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 436 | Num Epochs = 3 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 30,228,480 of 3,105,327,104 (0.97% trained)


Step,Training Loss
1,1.466400
2,1.452700
3,1.705300
4,1.678000
5,1.521800
6,1.590900
7,1.452800
8,1.436400
9,1.349300
10,1.379500


In [32]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

197.9702 seconds used for training.
3.3 minutes used for training.
Peak reserved memory = 8.641 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 21.844 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
# Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [33]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


["<|im_start|>user\nContinue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|im_end|>\n<|im_start|>assistant\n<think>\nD'accord, je dois continuer la suite de Fibonacci en commençant par 1, 1, 2, 3, 5, 8. La règle est que chaque nombre est la somme des deux précédents. Donc, après 8, le pro"]

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [34]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
    #{"role": "user", "content": "Quel est la fameuse grande tour à Paris?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

<|im_start|>user
Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|im_end|>
<|im_start|>assistant
<think>
D'accord, je dois continuer la suite de Fibonacci en commençant par 1, 1, 2, 3, 5, 8. La règle est que chaque nombre est la somme des deux précédents. Donc, après 8, le prochain nombre devrait être 8 + 5 = 13. Ensuite, 5 + 13 = 18, puis 13 + 18 = 31, 18 + 31 = 49, 31 + 49 = 80, 49 + 80 = 129, 


# Saving

In [35]:
#@title Create HF repository
from huggingface_hub import HfApi

if save_hf:
  hf_api = HfApi(token=hf_token)
  hf_api.create_repo(repo_id = hf_username + "/" + model_name, repo_type = "model", private = True, exist_ok = True)
  hf_api.create_repo(repo_id = hf_username + "/" + model_name + "-GGUF", repo_type = "model", private = True, exist_ok = True)


## Saving to float16 for VLLM (safetensors)

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [36]:
model.save_pretrained_merged(model_name, tokenizer)
if save_hf: model.push_to_hub_merged(hf_username + "/" + model_name, token = hf_token)

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Checking cache directory for required files...
Successfully copied all 2 files from cache to Claire-3B-v0.1.4.


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [00:29<00:00, 14.80s/it]


Unsloth: Merge process complete.


...

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ....4/model-00002-of-00002.safetensors:   0%|          | 4.28MB / 1.18GB            

Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [02:13<00:00, 66.71s/it]


Unsloth: Merge process complete.


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [37]:
model.save_pretrained_gguf(model_name, tokenizer, quantization_method = quantization_method)

if save_hf:
  for quant in quantization_method:
    hf_api.upload_file(
      path_or_fileobj=model_name + "." + quant.upper() + ".gguf",
      path_in_repo=model_name + "-" + quant.upper() + ".gguf",
      repo_id=hf_username + "/" + model_name + "-GGUF"
    )

Unsloth: Updating system package directories
Unsloth: Install GGUF and other packages
Unsloth GGUF:hf-to-gguf:Loading model: Claire-3B-v0.1.4
Unsloth GGUF:hf-to-gguf:Model architecture: SmolLM3ForCausalLM
Unsloth GGUF:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
Unsloth GGUF:hf-to-gguf:Exporting model...
Unsloth GGUF:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00001-of-00002.safetensors'
Unsloth GGUF:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> Q8_0, shape = {2048, 128256}
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00002-of-00002.safetensors'
Unsloth GGUF:hf-to-gguf:output_norm.weight,          torch.bfloat16 --> F32, shape = {2048}
Unsloth GGUF:hf-to-gguf:Set meta model
Unsloth GGUF:hf-to-gguf:Set model parameters
Unsloth GGUF:hf-to-gguf:gguf: context length = 65536
Unsloth GGUF:hf-to-gguf:gguf: embedding length = 2048
Unsloth GGUF:hf-to-gguf:gguf: fe

Unsloth: GGUF conversion:   0%|          | 0/100 [00:00<?, ?it/s]

Unsloth GGUF:hf-to-gguf:Model successfully exported to ./
Unsloth: Converted to Claire-3B-v0.1.4.Q8_0.gguf with size = 3.3G
Unsloth: Successfully saved GGUF to:
Claire-3B-v0.1.4.Q8_0.gguf
Unsloth GGUF:hf-to-gguf:Loading model: Claire-3B-v0.1.4
Unsloth GGUF:hf-to-gguf:Model architecture: SmolLM3ForCausalLM
Unsloth GGUF:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
Unsloth GGUF:hf-to-gguf:Exporting model...
Unsloth GGUF:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00001-of-00002.safetensors'
Unsloth GGUF:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> F16, shape = {2048, 128256}
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00002-of-00002.safetensors'
Unsloth GGUF:hf-to-gguf:output_norm.weight,          torch.bfloat16 --> F32, shape = {2048}
Unsloth GGUF:hf-to-gguf:Set meta model
Unsloth GGUF:hf-to-gguf:Set model parameters
Unsloth GGUF:hf-to-gguf:gguf: context 

Unsloth: GGUF conversion:   0%|          | 0/100 [00:00<?, ?it/s]

Unsloth GGUF:hf-to-gguf:Model successfully exported to ./
Unsloth: Converted to Claire-3B-v0.1.4.F16.gguf with size = 6.2G
Unsloth: Successfully saved GGUF to:
Claire-3B-v0.1.4.F16.gguf


...

New Data Upload                         : |          |  0.00B /  0.00B            

  Claire-3B-v0.1.4.Q8_0.gguf            :   1%|          | 25.2MB / 3.28GB            

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  Claire-3B-v0.1.4.F16.gguf             :   0%|          | 7.61MB / 6.16GB            